# Imports,  Settings and Paths

In [1]:
comment = 'bn_all_shuffle_sgd_lrd_095'

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd#
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [3]:
from Code.Models.c7o2h10_model import DeepPotential, train, normalize, backtransform
from Code.DataGeneration.saver import create_path

In [4]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
model_path = './ModelCheckpoints/c7o2h10/{}/'.format(comment)
create_path(model_path)

In [5]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [6]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   True


# Read the Data

In [7]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1
Y_npy, Y_min, Y_max = normalize(Y_npy)
X_npy.shape

(404000, 19, 72)

### Check for NaN values

In [8]:
use_ids = np.where(np.isnan(X_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [9]:
X_npy = X_npy[use_ids]
Y_npy = Y_npy[use_ids]
X_npy.shape

(404000, 19, 72)

In [10]:
X_npy = X_npy
Y_npy = Y_npy

### Split into test and train

In [11]:
X_train_npy, X_test_npy = X_npy[train_ids], X_npy[test_ids]
Y_train_npy, Y_test_npy = Y_npy[train_ids], Y_npy[test_ids]
print('X Train shape: {}\tX Test shape: {}\nY Train shape: {}\tY Test shape: {}'.format(X_train_npy.shape,
                                                                                        X_test_npy.shape,
                                                                                        Y_train_npy.shape,
                                                                                        Y_test_npy.shape))

X Train shape: (400000, 19, 72)	X Test shape: (4000, 19, 72)
Y Train shape: (400000,)	Y Test shape: (4000,)


### Shuffle the Dataset

In [12]:
shuffle = np.arange(X_train_npy.shape[0])
np.random.shuffle(shuffle)
X_train_npy = X_train_npy[shuffle]
Y_train_npy = Y_train_npy[shuffle]

### Create Pytorch Variables

In [13]:
if use_cuda:
    X_train = Variable(torch.Tensor(X_train_npy).cuda())
    Y_train = Variable(torch.Tensor(Y_train_npy).cuda(), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy).cuda())
    Y_test = Variable(torch.Tensor(Y_test_npy).cuda(), requires_grad=False)
else:
    X_train = Variable(torch.Tensor(X_train_npy))
    Y_train = Variable(torch.Tensor(Y_train_npy), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy))
    Y_test = Variable(torch.Tensor(Y_test_npy), requires_grad=False)

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU1 NVS 310 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


# Train Model

## Do the learning

In [ ]:
if use_cuda:
    init_deep_pot = DeepPotential().cuda()
else:
    init_deep_pot = DeepPotential()
init_optim = torch.optim.SGD(init_deep_pot.parameters(), lr=0.01)
deep_pot, optim = train(init_deep_pot, init_optim, X_train, Y_train,
                        X_test, Y_test,
                        300, 128, (0.01, 0.96, 1.5),
                        checkpoint_path=model_path, print_every=100, shuffle=True)

total: 0.0 %	current epoch: 3.2 %	loss: 0.056594	time estimate: 411.1 min
total: 0.0 %	current epoch: 6.4 %	loss: 0.036795	time estimate: 446.9 min
total: 0.0 %	current epoch: 9.6 %	loss: 0.027314	time estimate: 463.4 min
total: 0.0 %	current epoch: 12.8 %	loss: 0.021522	time estimate: 508.6 min
total: 0.1 %	current epoch: 16.0 %	loss: 0.017079	time estimate: 510.9 min
total: 0.1 %	current epoch: 19.2 %	loss: 0.014446	time estimate: 500.7 min
total: 0.1 %	current epoch: 22.4 %	loss: 0.012875	time estimate: 523.3 min
total: 0.1 %	current epoch: 25.6 %	loss: 0.011367	time estimate: 538.5 min
total: 0.1 %	current epoch: 28.8 %	loss: 0.010858	time estimate: 528.8 min
total: 0.1 %	current epoch: 32.0 %	loss: 0.009887	time estimate: 517.2 min
total: 0.1 %	current epoch: 35.2 %	loss: 0.009101	time estimate: 512.7 min
total: 0.1 %	current epoch: 38.4 %	loss: 0.008934	time estimate: 501.9 min
total: 0.1 %	current epoch: 41.6 %	loss: 0.008356	time estimate: 494.1 min
total: 0.1 %	current epoch: 

total: 1.2 %	current epoch: 48.0 %	loss: 0.003111	time estimate: 500.6 min
total: 1.2 %	current epoch: 51.2 %	loss: 0.003152	time estimate: 501.3 min
total: 1.2 %	current epoch: 54.4 %	loss: 0.00307	time estimate: 500.2 min
total: 1.2 %	current epoch: 57.6 %	loss: 0.00306	time estimate: 499.2 min
total: 1.2 %	current epoch: 60.8 %	loss: 0.003082	time estimate: 499.9 min
total: 1.2 %	current epoch: 64.0 %	loss: 0.003239	time estimate: 501.2 min
total: 1.2 %	current epoch: 67.2 %	loss: 0.003127	time estimate: 500.4 min
total: 1.2 %	current epoch: 70.4 %	loss: 0.003065	time estimate: 499.4 min
total: 1.2 %	current epoch: 73.6 %	loss: 0.003069	time estimate: 499.9 min
total: 1.3 %	current epoch: 76.8 %	loss: 0.002989	time estimate: 500.5 min
total: 1.3 %	current epoch: 80.0 %	loss: 0.003005	time estimate: 499.6 min
total: 1.3 %	current epoch: 83.2 %	loss: 0.003052	time estimate: 498.6 min
total: 1.3 %	current epoch: 86.4 %	loss: 0.003052	time estimate: 497.6 min
total: 1.3 %	current epoch:

total: 2.3 %	current epoch: 92.8 %	loss: 0.002662	time estimate: 500.2 min
total: 2.3 %	current epoch: 96.0 %	loss: 0.002646	time estimate: 500.3 min
total: 2.3 %	current epoch: 99.2 %	loss: 0.00265	time estimate: 499.7 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/bn_all_shuffle_sgd_lrd_095/epoch_6

total: 2.3 %	current epoch: 3.2 %	loss: 0.002608	time estimate: 499.6 min
total: 2.4 %	current epoch: 6.4 %	loss: 0.002612	time estimate: 499.4 min
total: 2.4 %	current epoch: 9.6 %	loss: 0.002623	time estimate: 498.8 min
total: 2.4 %	current epoch: 12.8 %	loss: 0.00264	time estimate: 498.3 min
total: 2.4 %	current epoch: 16.0 %	loss: 0.002582	time estimate: 497.7 min
total: 2.4 %	current epoch: 19.2 %	loss: 0.002635	time estimate: 497.2 min
total: 2.4 %	current epoch: 22.4 %	loss: 0.00264	time estimate: 496.7 min
total: 2.4 %	current epoch: 25.6 %	loss: 0.002579	time estimate: 496.1 min
total: 2.4 %	current epoch: 28.8 %	loss: 0.002592	time estimate: 495.9 min
total: 2.4 %	current e

total: 3.5 %	current epoch: 35.2 %	loss: 0.002414	time estimate: 492.4 min
total: 3.5 %	current epoch: 38.4 %	loss: 0.002418	time estimate: 492.0 min
total: 3.5 %	current epoch: 41.6 %	loss: 0.002424	time estimate: 491.7 min
total: 3.5 %	current epoch: 44.8 %	loss: 0.002409	time estimate: 491.5 min
total: 3.5 %	current epoch: 48.0 %	loss: 0.002387	time estimate: 491.1 min
total: 3.5 %	current epoch: 51.2 %	loss: 0.002408	time estimate: 490.7 min
total: 3.5 %	current epoch: 54.4 %	loss: 0.002481	time estimate: 491.0 min
total: 3.5 %	current epoch: 57.6 %	loss: 0.00248	time estimate: 491.0 min
total: 3.5 %	current epoch: 60.8 %	loss: 0.002483	time estimate: 490.7 min
total: 3.5 %	current epoch: 64.0 %	loss: 0.002446	time estimate: 491.0 min
total: 3.6 %	current epoch: 67.2 %	loss: 0.002428	time estimate: 490.8 min
total: 3.6 %	current epoch: 70.4 %	loss: 0.002508	time estimate: 490.3 min
total: 3.6 %	current epoch: 73.6 %	loss: 0.002495	time estimate: 490.4 min
total: 3.6 %	current epoch

total: 4.6 %	current epoch: 80.0 %	loss: 0.002357	time estimate: 482.3 min
total: 4.6 %	current epoch: 83.2 %	loss: 0.002329	time estimate: 482.0 min
total: 4.6 %	current epoch: 86.4 %	loss: 0.002317	time estimate: 481.7 min
total: 4.6 %	current epoch: 89.6 %	loss: 0.002288	time estimate: 481.8 min
total: 4.6 %	current epoch: 92.8 %	loss: 0.002333	time estimate: 482.0 min
total: 4.7 %	current epoch: 96.0 %	loss: 0.002273	time estimate: 481.7 min
total: 4.7 %	current epoch: 99.2 %	loss: 0.002275	time estimate: 482.0 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/bn_all_shuffle_sgd_lrd_095/epoch_13

total: 4.7 %	current epoch: 3.2 %	loss: 0.002268	time estimate: 482.3 min
total: 4.7 %	current epoch: 6.4 %	loss: 0.002261	time estimate: 482.4 min
total: 4.7 %	current epoch: 9.6 %	loss: 0.00231	time estimate: 482.1 min
total: 4.7 %	current epoch: 12.8 %	loss: 0.002264	time estimate: 482.2 min
total: 4.7 %	current epoch: 16.0 %	loss: 0.002272	time estimate: 482.5 min
total: 4.7 %	curren

...

total: 47.4 %	current epoch: 25.6 %	loss: 0.001932	time estimate: 268.9 min
total: 47.4 %	current epoch: 28.8 %	loss: 0.001929	time estimate: 268.8 min
total: 47.4 %	current epoch: 32.0 %	loss: 0.001954	time estimate: 268.8 min
total: 47.5 %	current epoch: 35.2 %	loss: 0.001981	time estimate: 268.7 min
total: 47.5 %	current epoch: 38.4 %	loss: 0.001971	time estimate: 268.7 min
total: 47.5 %	current epoch: 41.6 %	loss: 0.001964	time estimate: 268.6 min
total: 47.5 %	current epoch: 44.8 %	loss: 0.001951	time estimate: 268.5 min
total: 47.5 %	current epoch: 48.0 %	loss: 0.001942	time estimate: 268.5 min
total: 47.5 %	current epoch: 51.2 %	loss: 0.001908	time estimate: 268.4 min
total: 47.5 %	current epoch: 54.4 %	loss: 0.001916	time estimate: 268.4 min
total: 47.5 %	current epoch: 57.6 %	loss: 0.00194	time estimate: 268.3 min
total: 47.5 %	current epoch: 60.8 %	loss: 0.002004	time estimate: 268.3 min
total: 47.5 %	current epoch: 64.0 %	loss: 0.00197	time estimate: 268.2 min
total: 47.6 %	

total: 48.5 %	current epoch: 64.0 %	loss: 0.001932	time estimate: 262.8 min
total: 48.6 %	current epoch: 67.2 %	loss: 0.001919	time estimate: 262.8 min
total: 48.6 %	current epoch: 70.4 %	loss: 0.001957	time estimate: 262.7 min
total: 48.6 %	current epoch: 73.6 %	loss: 0.002005	time estimate: 262.7 min
total: 48.6 %	current epoch: 76.8 %	loss: 0.001967	time estimate: 262.6 min
total: 48.6 %	current epoch: 80.0 %	loss: 0.001944	time estimate: 262.6 min
total: 48.6 %	current epoch: 83.2 %	loss: 0.00193	time estimate: 262.6 min
total: 48.6 %	current epoch: 86.4 %	loss: 0.001974	time estimate: 262.5 min
total: 48.6 %	current epoch: 89.6 %	loss: 0.001897	time estimate: 262.4 min
total: 48.6 %	current epoch: 92.8 %	loss: 0.00191	time estimate: 262.4 min
total: 48.7 %	current epoch: 96.0 %	loss: 0.00191	time estimate: 262.3 min
total: 48.7 %	current epoch: 99.2 %	loss: 0.001918	time estimate: 262.3 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/bn_all_shuffle_sgd_lrd_095/epoch_145

total

saved checkpoint at: ./ModelCheckpoints/c7o2h10/bn_all_shuffle_sgd_lrd_095/epoch_148

total: 49.7 %	current epoch: 3.2 %	loss: 0.001904	time estimate: 257.0 min
total: 49.7 %	current epoch: 6.4 %	loss: 0.001971	time estimate: 257.0 min
total: 49.7 %	current epoch: 9.6 %	loss: 0.001948	time estimate: 256.9 min
total: 49.7 %	current epoch: 12.8 %	loss: 0.001963	time estimate: 256.8 min
total: 49.7 %	current epoch: 16.0 %	loss: 0.001904	time estimate: 256.8 min
total: 49.7 %	current epoch: 19.2 %	loss: 0.001896	time estimate: 256.7 min
total: 49.7 %	current epoch: 22.4 %	loss: 0.001946	time estimate: 256.7 min
total: 49.8 %	current epoch: 25.6 %	loss: 0.001932	time estimate: 256.6 min
total: 49.8 %	current epoch: 28.8 %	loss: 0.001993	time estimate: 256.6 min
total: 49.8 %	current epoch: 32.0 %	loss: 0.001917	time estimate: 256.6 min
total: 49.8 %	current epoch: 35.2 %	loss: 0.001954	time estimate: 256.5 min
total: 49.8 %	current epoch: 38.4 %	loss: 0.00197	time estimate: 256.4 min
total:

total: 50.8 %	current epoch: 38.4 %	loss: 0.002046	time estimate: 251.1 min
total: 50.8 %	current epoch: 41.6 %	loss: 0.001991	time estimate: 251.0 min
total: 50.8 %	current epoch: 44.8 %	loss: 0.001936	time estimate: 251.0 min
total: 50.8 %	current epoch: 48.0 %	loss: 0.001969	time estimate: 250.9 min
total: 50.8 %	current epoch: 51.2 %	loss: 0.001947	time estimate: 250.9 min
total: 50.8 %	current epoch: 54.4 %	loss: 0.001898	time estimate: 250.8 min
total: 50.9 %	current epoch: 57.6 %	loss: 0.001952	time estimate: 250.7 min
total: 50.9 %	current epoch: 60.8 %	loss: 0.001941	time estimate: 250.7 min
total: 50.9 %	current epoch: 64.0 %	loss: 0.001934	time estimate: 250.7 min
total: 50.9 %	current epoch: 67.2 %	loss: 0.001913	time estimate: 250.6 min
total: 50.9 %	current epoch: 70.4 %	loss: 0.001942	time estimate: 250.6 min
total: 50.9 %	current epoch: 73.6 %	loss: 0.001968	time estimate: 250.5 min
total: 50.9 %	current epoch: 76.8 %	loss: 0.001961	time estimate: 250.4 min
total: 50.9 

total: 51.9 %	current epoch: 76.8 %	loss: 0.001963	time estimate: 245.3 min
total: 51.9 %	current epoch: 80.0 %	loss: 0.001997	time estimate: 245.2 min
total: 51.9 %	current epoch: 83.2 %	loss: 0.001917	time estimate: 245.2 min
total: 52.0 %	current epoch: 86.4 %	loss: 0.001965	time estimate: 245.1 min
total: 52.0 %	current epoch: 89.6 %	loss: 0.001951	time estimate: 245.0 min
total: 52.0 %	current epoch: 92.8 %	loss: 0.001965	time estimate: 245.0 min
total: 52.0 %	current epoch: 96.0 %	loss: 0.001959	time estimate: 245.0 min
total: 52.0 %	current epoch: 99.2 %	loss: 0.001924	time estimate: 244.9 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/bn_all_shuffle_sgd_lrd_095/epoch_155

total: 52.0 %	current epoch: 3.2 %	loss: 0.001965	time estimate: 244.8 min
total: 52.0 %	current epoch: 6.4 %	loss: 0.002037	time estimate: 244.8 min
total: 52.0 %	current epoch: 9.6 %	loss: 0.001928	time estimate: 244.7 min
total: 52.0 %	current epoch: 12.8 %	loss: 0.001949	time estimate: 244.7 min
total

In [ ]:
#deep_pot = DeepPotential().cuda()
#deep_pot.load_state_dict(torch.load('./ModelCheckpoints/epoch_13'))
Y_result = deep_pot.forward(X_test)

In [ ]:
result = backtransform(np.array(Y_result.data.tolist()), Y_min, Y_max)

In [ ]:
test_labels = backtransform(np.array(Y_test), Y_min, Y_max)

In [ ]:
np.mean(np.abs(test_labels - result))

In [ ]:
test_labels[:10]

In [ ]:
result[:10]

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
test_result = np.squeeze(backtransform(test_model(X_test), Y_min, Y_max).detach().cpu().numpy())

In [ ]:
val_result = np.squeeze(backtransform(Y_test, Y_min, Y_max).detach().cpu().numpy())

In [ ]:
np.mean(np.abs(test_result - val_result))

In [ ]:
val_result[:10]

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)